###Reference
https://github.com/VICO-UoE/DatasetCondensation

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1513/Project_B_Supp/

Mounted at /content/drive
/content/drive/MyDrive/ECE1513/Project_B_Supp


In [ ]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=6d9fd72f85912e1a02b78999f0a9f1dae467335d512ea01d0deebb2636cf7270
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=7e99f996c83744f5a657a575f08a8513084fe9ceb1ad4cbdbb8b6e4d13773988
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [ ]:
import networks
import utils
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms,datasets
import tensorflow as tf
import pandas as pd
import torch.optim.lr_scheduler
import copy
from torchvision.utils import save_image


# Data Retrival

In [ ]:
# Here you need to unzip the folder either on drive or local directory
#!unzip /content/drive/MyDrive/ECE1513/Project_B_Supp/mhist_dataset/images.zip -d /content/drive/MyDrive/ECE1513/Project_B_Supp/mhist_dataset


In [ ]:
def create_directories(base_path, sub_folders):

    if not tf.io.gfile.exists(base_path):
        tf.io.gfile.mkdir(base_path)

    for folder in sub_folders:
        folder_path = f"{base_path}/{folder}"
        if not tf.io.gfile.exists(folder_path):
            tf.io.gfile.mkdir(folder_path)

base_paths = ['./mhist_dataset/train', './mhist_dataset/test','./mhist_dataset/augmentation']
sub_folders = ['HP', 'SSA']

for path in base_paths:
    create_directories(path, sub_folders)

In [ ]:
path = './mhist_dataset/annotations.csv'
df = pd.read_csv(path)
csv = dict(df)
image_num = len(csv.get('Image Name'))


for i in range(image_num):
  if csv.get('Partition')[i] == "train":
    if csv.get('Majority Vote Label')[i] == "HP":
      src = './mhist_dataset/images/' + csv.get('Image Name')[i]
      dst = './mhist_dataset/train/HP/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
    else:
      src = './mhist_dataset/images/' + csv.get('Image Name')[i]
      dst = './mhist_dataset/train/SSA/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
  else:
    if csv.get('Majority Vote Label')[i] == "HP":
      src = './mhist_dataset/images/' + csv.get('Image Name')[i]
      dst = './mhist_dataset/test/HP/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
    else:
      src = './mhist_dataset/images/' + csv.get('Image Name')[i]
      dst = './mhist_dataset/test/SSA/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)

In [ ]:
import matplotlib. image as image
from random import random

path = './mhist_dataset/annotations.csv'
df = pd.read_csv(path)
csv = dict(df)
image_num = len(csv.get('Image Name'))

for i in range(image_num):
    label = csv.get('Majority Vote Label')[i]
    src = './mhist_dataset/images/' + csv.get('Image Name')[i]
    img = image.imread(src)
    img_aug = tf.image.rot90(tf.image.random_flip_up_down(img), k=round(random()*3))

    base_name = os.path.splitext(csv.get('Image Name')[i])[0]
    extension = os.path.splitext(csv.get('Image Name')[i])[1]
    augmented_name = base_name + '_aug' + extension

    if label == "HP":
        # Save augmented image
        tf.keras.utils.save_img('./mhist_dataset/augmentation/HP/' + augmented_name, img_aug, data_format=None, file_format=None, scale=True)
        # Save original image
        tf.keras.utils.save_img('./mhist_dataset/augmentation/HP/' + csv.get('Image Name')[i], img, data_format=None, file_format=None, scale=True)
    elif label == "SSA":
        # Save augmented image
        tf.keras.utils.save_img('./mhist_dataset/augmentation/SSA/' + augmented_name, img_aug, data_format=None, file_format=None, scale=True)
        # Save original image
        tf.keras.utils.save_img('./mhist_dataset/augmentation/SSA/' + csv.get('Image Name')[i], img, data_format=None, file_format=None, scale=True)


# Baseline Training

##MHIST

In [ ]:
# initialize args
args = type('', (), {})()
args.lr_net = 0.01 # learning rate of the network
args.lr_img = 0.1 # learning rate of synthetic dataset
args.dsa_strategy = None
args.method = 'DC'
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.dsa_param = utils.ParamDiffAug()
args.dsa = True if args.method == 'DSA' else False
args.dc_aug_param = None

In [ ]:
IMG_SIZE = (32,32)
train_dir = './mhist_dataset/train'
test_dir = './mhist_dataset/test'
BATCH_SIZE=32


transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])
# Create the datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create the data loaders
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)


In [ ]:
def evaluate_dataset(it_eval, net, trainloader, testloader, args):
    net = net.to(args.device)
    criterion = nn.CrossEntropyLoss().to(args.device)
    optimizer = torch.optim.SGD(net.parameters(), lr=args.lr_net)
    net.train()
    # Cosine Annealing Scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epoch_train)
    # Train the network
    for ep in range(args.epoch_train):
        loss_train, acc_train = utils.epoch('train', trainloader, net, optimizer, criterion, args, aug=True)
        scheduler.step()
        print(f'{it_eval} Training Accuracy: {acc_train:.4f}, Training loss: {loss_train:.4f}')
    # Evaluate on the test set
    net.eval()
    loss_test, acc_test = utils.epoch('test', testloader, net, optimizer, criterion, args, aug=False)

    print(f'{it_eval} Training Accuracy: {acc_train:.4f}, Testing Accuracy: {acc_test:.4f}')

    return acc_train, acc_test

In [ ]:
args.model = 'ConvNetD5'
net = utils.get_network(args.model, channel=3, num_classes=2, im_size=IMG_SIZE).to(args.device)

In [ ]:
accuracy = []
it_eval = 1
args.epoch_train = 30
train_acc, test_acc = evaluate_dataset(it_eval, net, trainloader, testloader, args)
accuracy.append(test_acc)

# Print or process the accuracy as needed
print(f'Final Test Accuracy: {test_acc:.4f}')

1 Training Accuracy: 0.8423, Training loss: 0.4018
1 Training Accuracy: 0.9103, Training loss: 0.2715
1 Training Accuracy: 0.9306, Training loss: 0.2379
1 Training Accuracy: 0.9531, Training loss: 0.2083
1 Training Accuracy: 0.9605, Training loss: 0.1931
1 Training Accuracy: 0.9697, Training loss: 0.1677
1 Training Accuracy: 0.9853, Training loss: 0.1445
1 Training Accuracy: 0.9940, Training loss: 0.1158
1 Training Accuracy: 0.9926, Training loss: 0.1114
1 Training Accuracy: 0.9977, Training loss: 0.0950
1 Training Accuracy: 0.9982, Training loss: 0.0865
1 Training Accuracy: 0.9982, Training loss: 0.0798
1 Training Accuracy: 0.9986, Training loss: 0.0755
1 Training Accuracy: 0.9986, Training loss: 0.0727
1 Training Accuracy: 0.9991, Training loss: 0.0694
1 Training Accuracy: 0.9991, Training loss: 0.0665
1 Training Accuracy: 0.9995, Training loss: 0.0644
1 Training Accuracy: 0.9995, Training loss: 0.0625
1 Training Accuracy: 0.9995, Training loss: 0.0610
1 Training Accuracy: 0.9995, Tr

In [ ]:
# # Save the model's state dictionary
torch.save(net.state_dict(), 'conv_mhist.pth')

# net = utils.get_network(args.model, channel=3, num_classes=2, im_size=IMG_SIZE).to(args.device)
# net.load_state_dict(torch.load('conv7_mhist.pth'))
# net.eval()


In [ ]:
criterion = nn.CrossEntropyLoss().to(args.device)
optimizer = torch.optim.SGD(net.parameters(), lr=args.lr_net)
# Evaluate on the test set
loss_test, acc_test = utils.epoch('test', testloader, net, optimizer, criterion, args, aug=False)

print(f'Testing Accuracy: {acc_test:.4f}')

In [ ]:
from fvcore.nn import FlopCountAnalysis, parameter_count

def calculate_flops(model, input_size):
    inputs = torch.randn(input_size).to(args.device)
    flop_analysis = FlopCountAnalysis(model, inputs)
    return flop_analysis.total()

# Example usage
flops = calculate_flops(net, (1, 3, *IMG_SIZE))  # Adjust the input size as needed
print(f"FLOPs: {flops}")

FLOPs: 51589120


## MNIST

In [ ]:
args.model = 'ConvNet'
args.dataset = 'MNIST'
# Now you can use the dot notation
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, './data')

# Load and train network
net = utils.get_network(args.model, channel=channel, num_classes=num_classes, im_size=im_size).to(args.device)

In [ ]:
# Load dataset
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, './data')
args.batch_train = 32
# Load and train network
net = utils.get_network(args.model, channel=channel, num_classes=num_classes, im_size=im_size).to(args.device)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
accuracy = []
it_eval = 1
train_acc, test_acc = evaluate_dataset(it_eval, net, trainloader, testloader, args)
accuracy.append(test_acc)

# Print or process the accuracy as needed
print(f'Final Test Accuracy: {test_acc:.4f}')

1 Training Accuracy: 0.9627, Training loss: 0.1600
1 Training Accuracy: 0.9858, Training loss: 0.0526
1 Training Accuracy: 0.9890, Training loss: 0.0397
1 Training Accuracy: 0.9907, Training loss: 0.0326
1 Training Accuracy: 0.9922, Training loss: 0.0274
1 Training Accuracy: 0.9938, Training loss: 0.0235
1 Training Accuracy: 0.9945, Training loss: 0.0208
1 Training Accuracy: 0.9953, Training loss: 0.0186
1 Training Accuracy: 0.9958, Training loss: 0.0174
1 Training Accuracy: 0.9960, Training loss: 0.0166
1 Training Accuracy: 0.9960, Testing Accuracy: 0.9938
Final Test Accuracy: 0.9938


In [ ]:
# Save the model's state dictionary
torch.save(net.state_dict(), 'conv3_mnist.pth')

net = utils.get_network(args.model, channel=channel, num_classes=num_classes, im_size=im_size).to(args.device)
net.load_state_dict(torch.load('conv3_mnist.pth'))
net.eval()


In [ ]:
from fvcore.nn import FlopCountAnalysis, parameter_count

def calculate_flops(model, input_size):
    inputs = torch.randn(input_size).to(args.device)
    flop_analysis = FlopCountAnalysis(model, inputs)
    return flop_analysis.total()

# Example usage
flops = calculate_flops(net, (1, channel, *im_size))  # Adjust the input size as needed
print(f"FLOPs: {flops}")

FLOPs: 49246208


# Synthetic Data Training

In [ ]:
def data_condensation(outer_loop,inner_loop):
  # load real dataset
  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
  models = utils.get_eval_pool(args.eval_mode, args.model, args.model)

  # record accuracy of each model
  records = dict()

  for model in models:
    records[model] = []

  images_real = []
  labels_real = []

  #prepare individual image tensors for batch processing since PyTorch models typically expect inputs to have a batch dimension.
  images_real = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))] # converting to 3D tensor,(1,height,width)
  labels_real = [dst_train[i][1] for i in range(len(dst_train))]

  indices_class = [[] for c in range(num_classes)]
  for i, lab in enumerate(labels_real):
      indices_class[lab].append(i)
  images_real = torch.cat(images_real, dim=0).to(args.device)
  labels_real = torch.tensor(labels_real, dtype=torch.long, device=args.device) #torch.int64

  def get_images(c,n): # get n images from c classes
    idx_shuffle = np.random.permutation(indices_class[c])[:n]
    return images_real[idx_shuffle]

  ''' initialize the synthetic data '''
  if args.init == 'real':
      print('initialize synthetic data from random real images')
      label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]
      image_syn = torch.randn(size=(num_classes*args.ipc, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=args.device)
      for c in range(num_classes):
          image_syn.data[c*args.ipc:(c+1)*args.ipc] = get_images(c, args.ipc).detach().data
  else:
      print('initialize synthetic data from random noise')
      # args.ipc is the image per class
      image_syn = torch.randn(size=(num_classes*args.ipc, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=args.device)
      label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]

  # Training loop

  ''' training '''
  optimizer_img = torch.optim.SGD([image_syn, ], lr=args.lr_img, momentum=0.5) # optimizer for synthetic data, optimize the list of image_syn in this case
  optimizer_img.zero_grad()
  criterion = nn.CrossEntropyLoss().to(args.device)
  print('%s training begins'%utils.get_time())

  for it in range(args.Iteration+1):
    ''' Train synthetic data '''
    net = utils.get_network(args.model, channel, num_classes, im_size).to(args.device) # get model
    net.train()
    net_parameters = list(net.parameters())
    optimizer_net = torch.optim.SGD(net.parameters(), lr=args.lr_net)  # optimizer for synthetic data
    optimizer_net.zero_grad()
    #outer loop
    for k in range(outer_loop):
      loss = torch.tensor(0.0).to(args.device)
      for c in range(num_classes):
        batch_img_real = get_images(c, args.batch_real)
        batch_lab_real = torch.ones((batch_img_real.shape[0],), device=args.device, dtype=torch.long) * c
        img_syn = image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
        lab_syn = torch.ones((args.ipc,), device=args.device, dtype=torch.long) * c

        output_real = net(batch_img_real)
        loss_real = criterion(output_real, batch_lab_real)
        gw_real = torch.autograd.grad(loss_real, net_parameters)
        gw_real = list((_.detach().clone() for _ in gw_real))

        output_syn = net(img_syn)
        loss_syn = criterion(output_syn, lab_syn)
        gw_syn = torch.autograd.grad(loss_syn, net_parameters, create_graph=True)

        #compare gradient
        loss += utils.match_loss(gw_syn, gw_real, args)
      # update synthetic dataset with SGD
      optimizer_img.zero_grad()
      loss.backward()
      optimizer_img.step()
      torch.cuda.empty_cache()  # Clear cached memory

      ''' update network '''
      image_syn_train, label_syn_train = copy.deepcopy(image_syn.detach()), copy.deepcopy(label_syn.detach())  # avoid any unaware modification
      dst_syn_train = utils.TensorDataset(image_syn_train, label_syn_train)
      trainloader = torch.utils.data.DataLoader(dst_syn_train, batch_size=args.batch_train, shuffle=True, num_workers=0)


      # inner_loop to update model weight
      for il in range(inner_loop):
          utils.epoch('train', trainloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)

    # synthetic dataset evaluation
    if it == args.Iteration:
      args.epoch_eval_train = 200
      torch.cuda.empty_cache()
      for model in models:
        accuracy = []
        #loop over number of random model initialization
        for eval in range(args.num_eval):
          # load a network
          image_syn_eval, label_syn_eval = copy.deepcopy(image_syn.detach()), copy.deepcopy(label_syn.detach()) # avoid any unaware modification
          net = utils.get_network(model, channel, num_classes, im_size).to(args.device)
          _, acc_train, acc_test = utils.evaluate_synset(eval, net, image_syn_eval, label_syn_eval, testloader, args)
          accuracy.append(acc_test)

        print('Evaluate synthetic data on model: %s, mean accuracy = %.4f'%(model, np.mean(accuracy)))
        records[model] += accuracy
      ''' save and visualize'''
      save_name = os.path.join(args.save_path, 'vis_%s_%s_%s_%dipc_iter%d.png'%(args.method, args.dataset, args.model, args.ipc, it))
      #image_syn_vis = copy.deepcopy(image_syn.detach().cpu())
      if not tf.io.gfile.exists(args.save_path):
        tf.io.gfile.mkdir(args.save_path)
      save_image(image_syn, save_name, nrow=args.ipc)

  for model in models:
    accuracy = records[model]
    print("model = %s, accuracy= %.2f"%(args.model, np.mean(accuracy)*100))
  return image_syn, label_syn



## MNIST

In [ ]:
#args = type('', (), {})()
args.save_path = 'result'
args.dataset = 'MNIST'
args.eval_mode = 'S'
args.init = 'real'
args.data_path = './data'
args.dis_metric = 'ours'
args.device = 'cuda'
args.model = 'ConvNetD3'
args.num_eval=1

outer_loop = 50
inner_loop = 1
args.Iteration = 10
args.batch_train = 256
args.batch_real = 256
args.num_eval = 200
args.ipc = 10 # image per class
args.lr_net = 0.01 # learning rate of the network
args.lr_img = 0.1 # learning rate of synthetic dataset

image_syn, label_syn = data_condensation(outer_loop,inner_loop)

initialize synthetic data from random real images
[2023-11-28 02:15:42] training begins
[2023-11-28 02:20:05] Evaluate_00: epoch = 0200 train time = 2 s train loss = 0.004958 train acc = 1.0000, test acc = 0.9318
[2023-11-28 02:20:11] Evaluate_01: epoch = 0200 train time = 2 s train loss = 0.004847 train acc = 1.0000, test acc = 0.9348
[2023-11-28 02:20:16] Evaluate_02: epoch = 0200 train time = 2 s train loss = 0.004741 train acc = 1.0000, test acc = 0.9312
[2023-11-28 02:20:22] Evaluate_03: epoch = 0200 train time = 2 s train loss = 0.005147 train acc = 1.0000, test acc = 0.9324
[2023-11-28 02:20:27] Evaluate_04: epoch = 0200 train time = 2 s train loss = 0.004672 train acc = 1.0000, test acc = 0.9322
[2023-11-28 02:20:33] Evaluate_05: epoch = 0200 train time = 2 s train loss = 0.004716 train acc = 1.0000, test acc = 0.9333
[2023-11-28 02:20:39] Evaluate_06: epoch = 0200 train time = 2 s train loss = 0.005552 train acc = 1.0000, test acc = 0.9279
[2023-11-28 02:20:44] Evaluate_07: ep

In [ ]:
import io
torch.save(image_syn, 'image_syn_mnist.pt')
# Save to io.BytesIO buffer
buffer = io.BytesIO()
torch.save(image_syn, buffer)

torch.save(label_syn, 'label_syn_mnist.pt')
# Save to io.BytesIO buffer
buffer = io.BytesIO()
torch.save(label_syn, buffer)

## MHIST

In [ ]:
args = type('', (), {})()
args.save_path = 'result'
args.model = 'ConvNet'
args.dataset = 'MHIST'
args.eval_mode = 'S'
args.init = 'real'
args.data_path = './data'
args.dis_metric = 'ours'
args.device = 'cuda'
args.method = 'DC'
args.dsa_param = utils.ParamDiffAug()
args.dsa = True if args.method == 'DSA' else False
args.dc_aug_param = None

args.num_eval = 200 # number of random initialized weights
args.Iteration = 10 #Iteration
outer_loop = 50
inner_loop = 1
args.batch_train = 128
args.batch_real = 128
#args.num_eval = 3
args.lr_net = 0.01 # learning rate of the network
args.lr_img = 0.1 # learning rate of synthetic dataset
args.ipc = 50 # image per class

image_syn, label_syn = data_condensation(outer_loop,inner_loop)

initialize synthetic data from random real images
[2023-11-25 22:22:32] training begins
[2023-11-25 22:23:49] Evaluate_00: epoch = 0200 train time = 2 s train loss = 0.001953 train acc = 1.0000, test acc = 0.7277
[2023-11-25 22:23:56] Evaluate_01: epoch = 0200 train time = 3 s train loss = 0.001997 train acc = 1.0000, test acc = 0.7308
[2023-11-25 22:24:03] Evaluate_02: epoch = 0200 train time = 3 s train loss = 0.001832 train acc = 1.0000, test acc = 0.7349
Evaluate synthetic data on model: ConvNet, mean accuracy = 0.7311
model = ConvNet, accuracy= 73.11


In [ ]:
import io
torch.save(image_syn, 'image_syn.pt')
# Save to io.BytesIO buffer
buffer = io.BytesIO()
torch.save(image_syn, buffer)

torch.save(label_syn, 'label_syn.pt')
# Save to io.BytesIO buffer
buffer = io.BytesIO()
torch.save(label_syn, buffer)

# Gaussian noise Synthetic Data Training

##MNIST

In [ ]:
args.init = 'noise'
args.save_path = 'result'
args.dataset = 'MNIST'
args.eval_mode = 'S'
args.data_path = './data'
args.dis_metric = 'ours'
args.device = 'cuda'
args.model = 'ConvNetD3'
args.num_eval=100

outer_loop = 10
inner_loop = 50
args.Iteration = 10
args.batch_train = 256
args.batch_real = 256
args.num_eval = 1
args.ipc = 10 # image per class
args.lr_net = 0.01 # learning rate of the network
args.lr_img = 0.1 # learning rate of synthetic dataset

image_syn_noise_mnist, label_syn_noise_mnist = data_condensation(outer_loop,inner_loop)

initialize synthetic data from random noise
[2023-11-25 22:34:29] training begins
[2023-11-25 22:36:43] Evaluate_00: epoch = 0200 train time = 2 s train loss = 0.007150 train acc = 1.0000, test acc = 0.8563
Evaluate synthetic data on model: ConvNetD3, mean accuracy = 0.8563
model = ConvNetD3, accuracy= 85.63


## MHIST

In [ ]:
args = type('', (), {})()
args.save_path = 'result'
args.model = 'ConvNet'
args.dataset = 'MHIST'
args.eval_mode = 'S'
args.init = 'noise'
args.data_path = './data'
args.dis_metric = 'ours'
args.device = 'cuda'
args.method = 'DC'
args.dsa_param = utils.ParamDiffAug()
args.dsa = True if args.method == 'DSA' else False
args.dc_aug_param = None

args.num_eval = 200 # number of random initialized weights
args.Iteration = 10 #Iteration
outer_loop = 50
inner_loop = 1
args.batch_train = 128
args.batch_real = 128
#args.num_eval = 3
args.lr_net = 0.01 # learning rate of the network
args.lr_img = 0.1 # learning rate of synthetic dataset
args.ipc = 50 # image per class

image_syn_noise_mhist, label_syn_noise_mhist = data_condensation(outer_loop,inner_loop)

initialize synthetic data from random noise
[2023-11-25 22:43:46] training begins
[2023-11-25 22:45:03] Evaluate_00: epoch = 0200 train time = 2 s train loss = 0.001096 train acc = 1.0000, test acc = 0.6325
[2023-11-25 22:45:10] Evaluate_01: epoch = 0200 train time = 2 s train loss = 0.001192 train acc = 1.0000, test acc = 0.6448
[2023-11-25 22:45:18] Evaluate_02: epoch = 0200 train time = 3 s train loss = 0.001177 train acc = 1.0000, test acc = 0.6264
Evaluate synthetic data on model: ConvNet, mean accuracy = 0.6346
model = ConvNet, accuracy= 63.46


# Train Network from Scratch with Synthetic Dataset

In [ ]:
image_syn_noise_mhist = torch.load('image_syn.pt')
label_syn_noise_mhist = torch.load('label_syn.pt')

image_syn_noise_mnist = torch.load('image_syn_mnist.pt')
label_syn_noise_mnist = torch.load('label_syn_mnist.pt')

## MNIST

In [ ]:
args.dataset = 'MNIST'
args.epoch_train = 200
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
it_eval = args.num_eval
args.model = 'ConvNetD3'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)

images_train = image_syn_noise_mnist.to(args.device)
labels_train = label_syn_noise_mnist.to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)

it_eval = 1
train_acc, test_acc = evaluate_dataset(it_eval, net, trainloader, testloader, args)
#accuracy.append(test_acc)

# Print or process the accuracy as needed
print(f'Final Test Accuracy: {test_acc:.4f}')

1 Training Accuracy: 0.1100, Training loss: 2.3617
1 Training Accuracy: 0.2600, Training loss: 2.2205
1 Training Accuracy: 0.4600, Training loss: 2.1216
1 Training Accuracy: 0.6800, Training loss: 2.0376
1 Training Accuracy: 0.7300, Training loss: 1.9609
1 Training Accuracy: 0.7800, Training loss: 1.8892
1 Training Accuracy: 0.8100, Training loss: 1.8223
1 Training Accuracy: 0.8200, Training loss: 1.7597
1 Training Accuracy: 0.8400, Training loss: 1.7009
1 Training Accuracy: 0.8600, Training loss: 1.6454
1 Training Accuracy: 0.8700, Training loss: 1.5922
1 Training Accuracy: 0.9100, Training loss: 1.5414
1 Training Accuracy: 0.9300, Training loss: 1.4933
1 Training Accuracy: 0.9600, Training loss: 1.4471
1 Training Accuracy: 0.9700, Training loss: 1.4027
1 Training Accuracy: 0.9700, Training loss: 1.3603
1 Training Accuracy: 0.9700, Training loss: 1.3195
1 Training Accuracy: 0.9800, Training loss: 1.2803
1 Training Accuracy: 0.9800, Training loss: 1.2425
1 Training Accuracy: 0.9800, Tr

## MHIST

In [ ]:
args.dataset = 'MHIST'
args.epoch_train = 200
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
it_eval = args.num_eval
args.model = 'ConvNet'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)

images_train = image_syn_noise_mhist.to(args.device)
labels_train = label_syn_noise_mhist.to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)

it_eval = 1
train_acc, test_acc = evaluate_dataset(it_eval, net, trainloader, testloader, args)
#accuracy.append(test_acc)

# Print or process the accuracy as needed
print(f'Final Test Accuracy: {test_acc:.4f}')

1 Training Accuracy: 0.5000, Training loss: 0.6935
1 Training Accuracy: 0.5400, Training loss: 0.6776
1 Training Accuracy: 0.5300, Training loss: 0.6688
1 Training Accuracy: 0.5200, Training loss: 0.6657
1 Training Accuracy: 0.5100, Training loss: 0.6667
1 Training Accuracy: 0.5200, Training loss: 0.6726
1 Training Accuracy: 0.5000, Training loss: 0.6778
1 Training Accuracy: 0.5200, Training loss: 0.6838
1 Training Accuracy: 0.5000, Training loss: 0.6813
1 Training Accuracy: 0.5200, Training loss: 0.6781
1 Training Accuracy: 0.5000, Training loss: 0.6642
1 Training Accuracy: 0.5400, Training loss: 0.6539
1 Training Accuracy: 0.5100, Training loss: 0.6346
1 Training Accuracy: 0.5400, Training loss: 0.6211
1 Training Accuracy: 0.5200, Training loss: 0.6006
1 Training Accuracy: 0.5800, Training loss: 0.5858
1 Training Accuracy: 0.5800, Training loss: 0.5664
1 Training Accuracy: 0.5900, Training loss: 0.5522
1 Training Accuracy: 0.6300, Training loss: 0.5348
1 Training Accuracy: 0.7000, Tr

# Cross-architectural Generation

## MNIST

In [ ]:
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'AlexNet'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = image_syn_noise_mnist.to(args.device)
labels_train = label_syn_noise_mnist.to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
it_eval = 1
acc_train, acc_test = evaluate_dataset(it_eval,net,trainloader,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


1 Training Accuracy: 0.0900, Training loss: 2.3023
1 Training Accuracy: 0.0700, Training loss: 2.3007
1 Training Accuracy: 0.1300, Training loss: 2.2997
1 Training Accuracy: 0.1000, Training loss: 2.2983
1 Training Accuracy: 0.1400, Training loss: 2.2968
1 Training Accuracy: 0.1000, Training loss: 2.2953
1 Training Accuracy: 0.1700, Training loss: 2.2939
1 Training Accuracy: 0.1900, Training loss: 2.2933
1 Training Accuracy: 0.2000, Training loss: 2.2916
1 Training Accuracy: 0.2100, Training loss: 2.2903
1 Training Accuracy: 0.1600, Training loss: 2.2886
1 Training Accuracy: 0.1700, Training loss: 2.2872
1 Training Accuracy: 0.3400, Training loss: 2.2858
1 Training Accuracy: 0.3600, Training loss: 2.2843
1 Training Accuracy: 0.5200, Training loss: 2.2822
1 Training Accuracy: 0.3700, Training loss: 2.2805
1 Training Accuracy: 0.4600, Training loss: 2.2789
1 Training Accuracy: 0.1500, Training loss: 2.2776
1 Training Accuracy: 0.3200, Training loss: 2.2751
1 Training Accuracy: 0.3200, Tr

In [ ]:
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'VGG11'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = image_syn_noise_mnist.to(args.device)
labels_train = label_syn_noise_mnist.to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
it_eval = 1
acc_train, acc_test = evaluate_dataset(it_eval,net,trainloader,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


1 Training Accuracy: 0.0700, Training loss: 2.6498
1 Training Accuracy: 0.1200, Training loss: 3.6214
1 Training Accuracy: 0.0900, Training loss: 2.9937
1 Training Accuracy: 0.1000, Training loss: 4.3262
1 Training Accuracy: 0.0900, Training loss: 5.8690
1 Training Accuracy: 0.0700, Training loss: 4.3325
1 Training Accuracy: 0.0800, Training loss: 4.3367
1 Training Accuracy: 0.1400, Training loss: 3.6476
1 Training Accuracy: 0.1300, Training loss: 3.6192
1 Training Accuracy: 0.2500, Training loss: 2.4882
1 Training Accuracy: 0.1500, Training loss: 2.7832
1 Training Accuracy: 0.2000, Training loss: 2.3692
1 Training Accuracy: 0.5000, Training loss: 1.7920
1 Training Accuracy: 0.4700, Training loss: 1.7956
1 Training Accuracy: 0.5400, Training loss: 1.7039
1 Training Accuracy: 0.3400, Training loss: 2.5980
1 Training Accuracy: 0.6300, Training loss: 1.3730
1 Training Accuracy: 0.7400, Training loss: 1.0732
1 Training Accuracy: 0.6300, Training loss: 1.1796
1 Training Accuracy: 0.7700, Tr

In [ ]:
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'ResNet18'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = image_syn_noise_mnist.to(args.device)
labels_train = label_syn_noise_mnist.to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
it_eval = 1
acc_train, acc_test = evaluate_dataset(it_eval,net,trainloader,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


1 Training Accuracy: 0.0900, Training loss: 2.3758
1 Training Accuracy: 0.1100, Training loss: 2.2895
1 Training Accuracy: 0.1600, Training loss: 2.2426
1 Training Accuracy: 0.1500, Training loss: 2.2230
1 Training Accuracy: 0.3800, Training loss: 2.1272
1 Training Accuracy: 0.4600, Training loss: 2.0482
1 Training Accuracy: 0.3000, Training loss: 1.9973
1 Training Accuracy: 0.6800, Training loss: 1.8979
1 Training Accuracy: 0.6700, Training loss: 1.8343
1 Training Accuracy: 0.7800, Training loss: 1.7666
1 Training Accuracy: 0.6800, Training loss: 1.7353
1 Training Accuracy: 0.6200, Training loss: 1.8022
1 Training Accuracy: 0.7800, Training loss: 1.6075
1 Training Accuracy: 0.9000, Training loss: 1.5061
1 Training Accuracy: 0.7500, Training loss: 1.4969
1 Training Accuracy: 0.9300, Training loss: 1.4344
1 Training Accuracy: 0.9700, Training loss: 1.3313
1 Training Accuracy: 0.9700, Training loss: 1.3033
1 Training Accuracy: 0.8400, Training loss: 1.2829
1 Training Accuracy: 1.0000, Tr

##MHIST

In [ ]:
args.dataset = 'MHIST'
args.epoch_train = 20
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
#dst_test

Dataset ImageFolder
    Number of datapoints: 977
    Root location: ./mhist_dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=tensor([0.7378, 0.6486, 0.7752]), std=tensor([0.1889, 0.2307, 0.1640]))
           )

In [ ]:
args.epoch_train = 200

#image_syn_noise_mhist = torch.load('image_syn.pt')
#label_syn_noise_mhist = torch.load('label_syn.pt')

args.dataset = 'MHIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
#args.batch_train = 32

args.model = 'AlexNet'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
#net = AlexNet(channel=channel, num_classes=num_classes)
it_eval = args.num_eval

images_train = image_syn_noise_mhist.to(args.device)
labels_train = label_syn_noise_mhist.to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
it_eval = 1
acc_train, acc_test = evaluate_dataset(it_eval,net,trainloader,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


1 Training Accuracy: 0.3700, Training loss: 0.6938
1 Training Accuracy: 0.5000, Training loss: 0.6933
1 Training Accuracy: 0.5200, Training loss: 0.6933
1 Training Accuracy: 0.5000, Training loss: 0.6932
1 Training Accuracy: 0.5000, Training loss: 0.6930
1 Training Accuracy: 0.5300, Training loss: 0.6928
1 Training Accuracy: 0.5000, Training loss: 0.6927
1 Training Accuracy: 0.5000, Training loss: 0.6925
1 Training Accuracy: 0.5000, Training loss: 0.6924
1 Training Accuracy: 0.5000, Training loss: 0.6922
1 Training Accuracy: 0.6200, Training loss: 0.6920
1 Training Accuracy: 0.5100, Training loss: 0.6922
1 Training Accuracy: 0.5000, Training loss: 0.6918
1 Training Accuracy: 0.5700, Training loss: 0.6918
1 Training Accuracy: 0.5100, Training loss: 0.6915
1 Training Accuracy: 0.5000, Training loss: 0.6916
1 Training Accuracy: 0.5000, Training loss: 0.6912
1 Training Accuracy: 0.5300, Training loss: 0.6910
1 Training Accuracy: 0.6300, Training loss: 0.6908
1 Training Accuracy: 0.6600, Tr

In [ ]:
args.dataset = 'MHIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'VGG11'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = image_syn_noise_mhist.to(args.device)
labels_train = label_syn_noise_mhist.to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
it_eval = 1
acc_train, acc_test = evaluate_dataset(it_eval,net,trainloader,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


1 Training Accuracy: 0.5200, Training loss: 1.8551
1 Training Accuracy: 0.4000, Training loss: 4.0590
1 Training Accuracy: 0.4400, Training loss: 3.5744
1 Training Accuracy: 0.4400, Training loss: 2.6629
1 Training Accuracy: 0.5200, Training loss: 3.0567
1 Training Accuracy: 0.4600, Training loss: 2.1445
1 Training Accuracy: 0.5000, Training loss: 1.5006
1 Training Accuracy: 0.7600, Training loss: 0.6437
1 Training Accuracy: 0.5200, Training loss: 2.5084
1 Training Accuracy: 0.4600, Training loss: 1.6501
1 Training Accuracy: 0.5200, Training loss: 1.2401
1 Training Accuracy: 0.3800, Training loss: 2.0906
1 Training Accuracy: 0.8800, Training loss: 0.4712
1 Training Accuracy: 0.5900, Training loss: 0.6321
1 Training Accuracy: 0.5000, Training loss: 0.9168
1 Training Accuracy: 0.6700, Training loss: 0.4612
1 Training Accuracy: 0.4800, Training loss: 1.4252
1 Training Accuracy: 0.4800, Training loss: 1.4983
1 Training Accuracy: 0.5200, Training loss: 2.3512
1 Training Accuracy: 0.5900, Tr

In [ ]:
args.dataset = 'MHIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'ResNet18'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = image_syn_noise_mhist.to(args.device)
labels_train = label_syn_noise_mhist.to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
it_eval = 1
acc_train, acc_test = evaluate_dataset(it_eval,net,trainloader,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


1 Training Accuracy: 0.5600, Training loss: 0.7176
1 Training Accuracy: 0.6000, Training loss: 1.0314
1 Training Accuracy: 0.5000, Training loss: 0.8791
1 Training Accuracy: 0.6500, Training loss: 0.5975
1 Training Accuracy: 0.8300, Training loss: 0.5322
1 Training Accuracy: 1.0000, Training loss: 0.4831
1 Training Accuracy: 1.0000, Training loss: 0.4537
1 Training Accuracy: 0.6800, Training loss: 0.4752
1 Training Accuracy: 0.7600, Training loss: 0.5325
1 Training Accuracy: 1.0000, Training loss: 0.3408
1 Training Accuracy: 0.8500, Training loss: 0.3780
1 Training Accuracy: 0.9900, Training loss: 0.2583
1 Training Accuracy: 1.0000, Training loss: 0.2598
1 Training Accuracy: 1.0000, Training loss: 0.2198
1 Training Accuracy: 1.0000, Training loss: 0.2229
1 Training Accuracy: 1.0000, Training loss: 0.1705
1 Training Accuracy: 1.0000, Training loss: 0.1324
1 Training Accuracy: 1.0000, Training loss: 0.1317
1 Training Accuracy: 1.0000, Training loss: 0.1046
1 Training Accuracy: 1.0000, Tr